# SEYFERT Interactive Notebook

Imports

In [ ]:
import time

t_begin = time.time()

from pathlib import Path
import numpy as np
import importlib
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import datetime
import re
import sys
import copy
import json
import itertools
import pickle

from seyfert.utils import general_utils, filesystem_utils, formatters
from seyfert.utils.tex_utils import TeXTranslator

plt.style.use("plt_params.mplstyle")

from seyfert.config.forecast_config import ForecastConfig
from seyfert.utils.workspace import WorkSpace
from seyfert.cosmology import cosmology
from seyfert.cosmology import redshift_density
from seyfert.cosmology import c_ells

from seyfert.main import cl_core
from seyfert.main import cl_derivative_core
from seyfert.main import fisher_core

from seyfert import plot_utils

transl = TeXTranslator()

Logging: if you want full logs set do_full_log = True

In [ ]:
do_full_log = False

import logging
logger = logging.getLogger()

if do_full_log:
    general_utils.configure_logger(logger)

## Configurations

SEYFERT configuration files are all written in JSON format, and are essentially of two types: 

* ForecastConfig: the master configuration for the forecast. It is associated to a ForecastConfig class defined in `seyfert/config/forecast_config.py`
* MainConfig: the configuration for the main script do the single computation tasks, which are:
    * computation of the power spectra;
    * computation of the angular power spectra;
    * computation of the derivatives of the angular power spectra;
    * computation of the fisher matrices.

First of all we do the workspace setup. The necessary input data are:

* Configuration files: these are the above mentioned JSON files. Some example files are stored inside the directory `input/config`, and are written in the cell below.
* Input data files: these are auxiliary files storing the configuration of galaxy redshift densities or galaxy bias. The example files are here stored in `input/data`.

In [ ]:
shotnoise_sp_reduced = True

input_data_dir = Path("/home/euclid/spectrophoto/input_data/")

src_input_files = {
    'forecast': "/home/euclid/spectrophoto/config_files/latest/test_photo_4_bins/test_gcph_4_bins_gcsp_4_bins.json",
    'PowerSpectrum': "input/config/power_spectrum_config.json",
    'Angular': "input/config/angular_config.json",
    'Derivative': "input/config/derivative_config.json",
    'Fisher': "input/config/fisher_config.json"
}

fcfg = ForecastConfig(input_file=src_input_files['forecast'], input_data_dir=input_data_dir)
fcfg.loadPhysicalParametersFromJSONConfig()
phys_pars = fcfg.phys_pars

if shotnoise_sp_reduced:
    fcfg.shot_noise_file = f"/home/euclid/spectrophoto/input_data/shot_noise/shotnoise_{fcfg.n_sp_bins}_sp_bins_gcph_only_bins_spectro_range_shotnoise_gcsp_reduced.h5"
    fcfg.synthetic_opts['shot_noise_sp_reduced'] = shotnoise_sp_reduced


In [ ]:
test = False

if test:
    rundir_name = "run_test"
else:
    now = datetime.datetime.now()
    rundir_name = f"run_{fcfg.getConfigID()}_{formatters.datetime_str_format(now)}"

ws = WorkSpace(rundir_name)
ws.run_dir.mkdir(exist_ok=True)

ws.createInputFilesDir(src_input_files=src_input_files, phys_pars=phys_pars, input_data_dir=input_data_dir)

main_configs = ws.getTasksJSONConfigs()

## Fiducial Cosmology

In this forecast we work in a $w_0 w_a \rm CDM$ flat cosmology, therefore the Hubble parameter is given by

\begin{equation}
H(z)=H_{0} \sqrt{\Omega_{\mathrm{m}}(1+z)^{3}+\left(1-\Omega_{\mathrm{m}}\right)(1+z)^{3\left(1+w_{0}+w_{a}\right)} \exp \left(-3 w_{a} \frac{z}{1+z}\right)}
\end{equation}

In following cell the fiducial cosmology is loaded from file. Technically SEYFERT is also capable to call CAMB or CLASS to compute the matter power spectrum on the fly, but here in order to save time we load the already computed power spectra from disk.

In [ ]:
pmm_dir = Path("/ssd860/euclid_forecasts/spectrophoto/powerspectra/istf_pmms/")
#pmm_dir = Path("/Users/lucapaganin/spectrophoto/powerspectra/istf_pmms/")

if not ws.pmm_dir.exists():
    ws.symlinkToExternalDirs({
        "PowerSpectrum": pmm_dir,
        "Angular": "run_scenario_optimistic__n_sp_bins_4__gcph_only_bins_in_spectro_range_2021-06-27T19-32-42/Angular/",
        "Derivative": "run_scenario_optimistic__n_sp_bins_4__gcph_only_bins_in_spectro_range_2021-06-27T19-32-42/Derivative/"
    })

fid_cosmo = cosmology.Cosmology.fromHDF5(ws.getPowerSpectrumFile(dvar='central', step=0))
fid_cosmo.evaluateOverRedshiftGrid()

## Redshift densities

In [ ]:
if not ws.niz_file.is_file():
    densities = {}

    for probe, pcfg in fcfg.probe_configs.items():
        densities[probe] = redshift_density.RedshiftDensity.fromHDF5(pcfg.density_init_file)
        densities[probe].setUp()
        densities[probe].evaluate(z_grid=fid_cosmo.z_grid)
        densities[probe].evaluateSurfaceDensity()

    redshift_density.save_densities_to_file(densities=densities, file=ws.niz_file)
else:
    densities = redshift_density.load_densities_from_file(file=ws.niz_file)

### Photo-z densities

Here we plot the $\rm GC_{ph}$ galaxy densities. The tomographic bins used are 10 equi-populated bins in the range $z \in [0.001, 2.5]$. The bin edges are the following

\begin{equation}
z_{i}=\{0.0010,0.42,0.56,0.68,0.79,0.90,1.02,1.15,1.32,1.58,2.50\}
\end{equation}

In [ ]:
nph = densities["PhotometricGalaxy"]

for i in range(nph.n_bins):
    plt.plot(nph.z_grid, nph.norm_density_iz[i], label=f'{i+1}')

plt.xlabel("$z$", fontsize=22)
plt.legend(bbox_to_anchor=[1, 0.38], title="bin index")
plt.title(r"$\rm GC_{ph}$")
    
del nph

### Spectro-z densities

Here we plot the $\rm GC_{sp}$ galaxy densities. The baseline tomographic bins used for spectro-z are the 4 bins used for the power spectrum analysis of IST:F

\begin{equation}
z_{i} = \{0.9, 1.1, 1.3, 1.5, 1.8\}
\end{equation}

In [ ]:
nsp = densities["SpectroscopicGalaxy"]

for i in range(nsp.n_bins):
    plt.plot(nsp.z_grid, nsp.norm_density_iz[i], label=f'{i+1}')

plt.xlabel("$z$")
plt.legend(bbox_to_anchor=[1, 0.70], title="bin index")
plt.title(r"$\rm GC_{sp}$")
    
del nsp

## Angular power spectra


In [ ]:
ws.cl_dir.mkdir(exist_ok=True)

fid_cl_file = ws.cl_dir / "dvar_central_step_0" / "cls_fiducial.h5"

if not fid_cl_file.is_file():

    fid_cls = cl_core.compute_cls(cosmology=fid_cosmo, phys_pars=phys_pars, densities=densities, 
                                  forecast_config=fcfg, angular_config=main_configs['Angular'])

    fid_cl_file.parent.mkdir(exist_ok=True, parents=True)

    fid_cls.saveToHDF5(fid_cl_file)
else:
    fid_cls = c_ells.AngularCoefficientsCollector.fromHDF5(fid_cl_file)

del fid_cl_file

### Plot the weight functions

Here we plot the weight functions

In [ ]:
fig, axs = plot_utils.cl_plot.plot_weight_funcs(fid_cls, cosmo=fid_cosmo, phys_pars=phys_pars)


### Plot auto-correlation angular power spectra

Here we plot the diagonals of the auto-correlations $C(\ell)$'s

In [ ]:
for kind in ['phph', 'spsp', 'wlwl']:
    fig = plt.figure()
    cl = fid_cls[kind]
    ells = cl.l_bin_centers

    for i in range(cl.n_i):
        plt.loglog(ells, ells*(ells+1)*cl.c_lij[:, i, i], label=i)

    plt.legend(bbox_to_anchor=[1.0, 0.4], title="bin index")
    plt.title(r"$C^{\rm %s}_{ii}(\ell)$" % kind)
    plt.xlabel(r"$\ell$", fontsize=22)
    plt.ylabel(r"$\ell (\ell+1) C(\ell)$", fontsize=22)

## Derivatives of angular power spectra

Compute $C(\ell)$ derivatives with respect to the free parameters, which may be cosmological or nuisance. The derivatives are computed with the SteM® algorithm (credits to Stefano Camera).


First of all set the free parameters

In [ ]:
#free_pars = ["w0", "wa", "Omm"]
free_pars = list(phys_pars.free_physical_parameters.keys())

for name in phys_pars:
    if name not in free_pars:
        phys_pars[name].is_free_parameter = False
    else:
        phys_pars[name].is_free_parameter = True

Do the computation

In [ ]:
cl_ders_file = ws.der_dir / "cl_ders.pickle"

if not ws.der_dir.is_dir():
    ws.der_dir.mkdir()
    cl_ders_file = ws.der_dir / "cl_ders.pickle"

    data = {
        "fid_cls": fid_cls,
        "ws": ws,
        "phys_pars": phys_pars,
        "densities": densities,
        "forecast_config": fcfg,
        "angular_config": main_configs['Angular'],
        "fiducial_cosmology": fid_cosmo
    }
    
    ti = time.time()
    ders_dict = {}
    n_params = len(free_pars)

    for i, dvar in enumerate(phys_pars.free_physical_parameters):
        t1 = time.time()
        print(f"{'#'*40} Computing cl derivatives w.r.t. {dvar}: {i+1}/{n_params} {'#'*40}")
        ders_dict[dvar] = cl_derivative_core.compute_cls_derivatives_wrt(dvar, **data)
        t2 = time.time()
        print(f"Elapsed time: {formatters.string_time_format(t2 - t1)}")


    tf = time.time()
    print("")
    print(f"Cl derivatives total elapsed time: {formatters.string_time_format(tf - ti)}")

    with open(cl_ders_file, mode="wb") as f:
        pickle.dump(ders_dict, f)
else:
    with open(cl_ders_file, mode="rb") as f:
        ders_dict = pickle.load(f)
        

### Plot of derivatives

In [ ]:
dcl_phph = ders_dict['w0'].dcl_dict['PhotometricGalaxy_PhotometricGalaxy']

for i in range(dcl_phph.dc_lij.shape[1]):
    plt.plot(dcl_phph.l_bin_centers, dcl_phph.dc_lij[:, i, i], label=i+1)

plt.legend(bbox_to_anchor=[1.0, 0.4], title="bin index")
plt.title(r"Derivative of $C^{\rm phph}_{ii}(\ell)$ w.r.t. to $w_0$")
plt.xscale('log')
plt.xlabel(r"$\ell$", fontsize=22)
plt.ylabel(r"$\frac{\partial C^{\rm phph}_{ii}(\ell)}{\partial w_0}$", rotation=0, fontsize=22, labelpad=50)

## Compute and save Delta Cls

Compute the $\Delta C(\ell)$'s. These are simply defined as

\begin{equation}
\Delta C^{AB}_{ij}(\ell) = \frac{1}{\sqrt{f_{\rm sky}\Delta\ell}} \left[ C^{AB}_{ij}(\ell) + N^{AB}_{ij}(\ell) \right]
\end{equation}

where $N^{AB}_{ij}(\ell)$ is the Poisson shot noise, and $f_{\rm sky}$ is the sky fraction covered by Euclid. The sky fraction is computed assuming an observed sky area of $15000 \, \mathrm{deg}^2$, and therefore since the full sky is approximately $41252.961 \, \mathrm{deg}^2$ we have

\begin{equation}
f_{\rm sky} \simeq 0.36361
\end{equation}

The covariance matrix for the $C(\ell)$'s is calculated as

\begin{equation}
\mathrm{Cov}[ C^{AB}_{ij}(\ell), C^{CD}_{km}(\ell')] = \frac{\delta_{\ell\ell'}^{\rm K}}{2\ell + 1} 
\left[
\Delta C^{AC}_{ij}(\ell)\Delta C^{BD}_{ij}(\ell') + \Delta C^{AD}_{ij}(\ell) \Delta C^{BC}_{ij}(\ell')
\right]
\end{equation}

where $\delta_{\ell\ell'}^{\rm K}$ is the Kronecker delta.

The following cell computes and saves on disk the $\Delta C^{AB}(\ell)$'s for each combination of the probe indices $A$ and $B$.

In [ ]:
if ws.delta_cls_file.is_symlink():
    ws.delta_cls_file.unlink()

if ws.delta_cls_file.is_file():
    delta_cls = cl_core.DeltaClCollection.fromHDF5(ws.delta_cls_file)
else:
    delta_cls = cl_core.compute_delta_cls(ws)
    delta_cls.saveToHDF5(ws.delta_cls_file)    

## Fisher

In [ ]:
ws.base_fishers_dir.mkdir(exist_ok=True)

fisher_input_data = {
    "outdir": ws.base_fishers_dir,
    "ws": ws,
    "phys_pars": phys_pars,
    "delta_cls": delta_cls,
    "dcoll_dvar_dict": ders_dict
}

auto_fishers = fisher_core.compute_and_save_fishers(["phph", "spsp", "wlwl"], ws.base_fishers_dir, ws, phys_pars, delta_cls, ders_dict)

Now compute some combinations

In [ ]:
from seyfert.fisher.fisher_utils import load_selected_data_vectors

In [ ]:
datavectors = load_selected_data_vectors()

In [ ]:
brief_str_datavectors = [dv.toBriefString() for dv in datavectors]

In [ ]:
fishers = fisher_core.compute_and_save_fishers(brief_str_datavectors, ws.base_fishers_dir, ws, phys_pars, delta_cls, ders_dict)

Load IST:F Pk fisher matrix

In [ ]:
from seyfert.fisher.fisher_matrix import FisherMatrix
from seyfert.fisher.final_results_core import create_final_results


In [ ]:
f_gcsp_pk = FisherMatrix.from_ISTfile(filesystem_utils.get_ist_gcsp_pk_fisher_file(fcfg.scenario))


f_gcsp_pk.writeToFile(outfile=ws.base_fishers_dir / "fisher_IST_gcsp_pk.hdf5")

In [ ]:
create_final_results(rundir=ws.run_dir, outdir_name="final_results")

## Some contour plots

Plot Fisher contours

In [ ]:
from seyfert.fisher.fisher_analysis import FisherAnalysis
from seyfert.fisher.fisher_plot import FisherPlotter

full_analysis = FisherAnalysis.fromFishersDir(ws.getResultsDir(), params=phys_pars)

cmap = matplotlib.colors.ListedColormap(["dodgerblue", "forestgreen", "gold"])
pars_to_plot=["Omm", "h", "w0", "wa"]

In [ ]:
fishers = [
    "[GCph]", "[GCsp]", "[GCph+GCsp]", "[GCph+GCsp+XC(GCph,GCsp)]"
]

analysis = full_analysis.getSubsetAnalysis(fishers)

analysis.evaluateMarginalizedErrors()

### 6x2pt

In [ ]:
analysis.name = "test"

plotter = FisherPlotter(pars_to_plot=pars_to_plot, fisher_analysis=analysis)
plotter.setPlotConfig(config_file="input/config/results_config.json")
plotter.setParametersPlotRanges()

fig, axs = plotter.makeTriangularPlot()

In [ ]:
Nph_tot = densities['PhotometricGalaxy'].computeTotalGalaxyNumber()
Nsp_tot = densities['SpectroscopicGalaxy'].computeTotalGalaxyNumber()

delta_cls.single_blocks['SpectroscopicGalaxy_SpectroscopicGalaxy'].noise_lij *= (Nsp_tot / Nph_tot)

In [ ]:
delta_cls.writeShotNoiseToFile(f"shotnoise_{fcfg.n_sp_bins}_sp_bins_gcph_only_bins_spectro_range_shotnoise_gcsp_reduced.h5")

In [ ]:
t_end = time.time()

print(f"Total notebook run time: {formatters.string_time_format(t_end - t_begin)}")